# **VIRTUAL RACE ENGINEER**

In [31]:
import fastf1 as ff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1.plotting
fastf1.plotting.setup_mpl()
import fastf1.api as fap
fastf1.Cache.enable_cache(r"D:\Prabhu\SEM 7\F1 Data")
import re
import datetime as dt
import fastf1.mvapi as famp


### LOADING SESSION

In [32]:
session = ff.get_session(2023,'Dutch','R')
session.load()

core           INFO 	Loading data for Dutch Grand Prix - Race [v3.3.7]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '14', '10', '11', '55', '44', '4', '23', '81', '31', '18', '27', '40', '77', '22', '20', '63', '24', '16', '2']


In [33]:
laps=session.laps
drivers=laps['Driver'].unique()

### FETCHING TELEMETRY DATA

In [48]:
total_telemetry=[]
for i in drivers:
    drivers_laps=laps.pick_driver(i)
    driver_telemetry=drivers_laps.get_car_data()
    
    driver_telemetry['Driver']=i
    total_telemetry.append(driver_telemetry)
telemetry_data=pd.concat(total_telemetry,ignore_index=True)
telemetry_data


,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Driver
0,2023-08-27 13:03:05.416,10093,0,1,15,False,1,car,0 days 00:00:00.082000,0 days 01:02:05.042000,VER
1,2023-08-27 13:03:05.616,10063,0,1,15,False,1,car,0 days 00:00:00.282000,0 days 01:02:05.242000,VER
2,2023-08-27 13:03:05.897,8517,1,1,15,False,1,car,0 days 00:00:00.563000,0 days 01:02:05.523000,VER
3,2023-08-27 13:03:06.097,7117,11,1,15,False,1,car,0 days 00:00:00.763000,0 days 01:02:05.723000,VER
4,2023-08-27 13:03:06.497,4317,24,1,16,False,1,car,0 days 00:00:01.163000,0 days 01:02:06.123000,VER
...,...,...,...,...,...,...,...,...,...,...,...
601328,2023-08-27 15:27:25.367,11670,294,7,99,False,1,car,0 days 02:24:20.033000,0 days 03:26:24.993000,PIA
601329,2023-08-27 15:27:25.527,11699,295,7,99,False,1,car,0 days 02:24:20.193000,0 days 03:26:25.153000,PIA
601330,2023-08-27 15:27:25.687,11742,295,7,99,False,1,car,0 days 02:24:20.353000,0 days 03:26:25.313000,PIA
601331,2023-08-27 15:27:26.047,11758,296,7,99,False,1,car,0 days 02:24:20.713000,0 days 03:26:25.673000,PIA


### FETCHING WEATHER,TYRE,LAP AND POSITION DATA 

In [43]:
weather_data=pd.DataFrame(fap.weather_data(session.api_path))
tyre_data = laps[['Driver', 'LapNumber', 'Compound', 'FreshTyre','TyreLife']]
lap_data=laps[['Driver','LapNumber','LapTime','Sector1Time','Sector2Time','Sector3Time','IsAccurate']]
position_data=laps[['Driver','LapNumber','Position']]
weather_data

req            INFO 	Using cached data for weather_data


,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,0 days 00:00:46.338000,17.4,68.0,1007.4,False,25.5,196,2.7
1,0 days 00:01:46.337000,17.4,67.0,1007.5,False,25.5,193,3.2
2,0 days 00:02:46.336000,17.4,66.0,1007.4,False,25.5,177,3.4
3,0 days 00:03:46.335000,17.5,67.0,1007.4,False,25.6,184,2.4
4,0 days 00:04:46.349000,17.5,67.0,1007.5,False,25.6,186,2.9
...,...,...,...,...,...,...,...,...
204,0 days 03:24:47.183000,15.7,79.0,1008.2,True,19.0,176,3.0
205,0 days 03:25:47.182000,15.8,77.0,1008.2,True,19.1,161,3.0
206,0 days 03:26:47.181000,15.8,77.0,1008.2,True,19.1,163,2.7
207,0 days 03:27:47.180000,15.7,74.0,1008.4,False,18.9,192,3.2


### FETCHING RACE CONTROL MESSAGES

In [36]:
rcm_data = pd.DataFrame(fap.race_control_messages(session.api_path))
msg=rcm_data[['Time','Status','Message']]

#Filter for Yellow, Red, VSC, SC 
yellow_flags_key=['YELLOW','DOUBLE YELLOW']
red_flags_key=['RED']
vsc_key=['VSC','VIRTUAL SAFETY CAR']
sc_key=['SC','SAFETY CAR']

#Searching using the keyword in REGEX
yellow_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in yellow_flags_key))]
red_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in red_flags_key))]
vsc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in vsc_key))]
sc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in sc_key))]


req            INFO 	Using cached data for race_control_messages


### FETCHING LAP DATA (PITS STOPS, DRIVER POSITION ETC.)

In [37]:
lapsdata, streamdata = fap.timing_data(session.api_path)
# pd.set_option('display.max_columns', None)
lapsdata=pd.DataFrame(lapsdata)
streamdata=pd.DataFrame(streamdata)
combined=pd.concat([lapsdata,streamdata])
laps_data=combined[['Driver','LapTime','NumberOfLaps','NumberOfPitStops','PitOutTime','PitInTime','Position','GapToLeader','IntervalToPositionAhead']]
laps_data['PitInTimeSec']=laps_data['PitInTime'].dt.total_seconds()
laps_data['PitOutTimeSec']=laps_data['PitOutTime'].dt.total_seconds()
laps_data

req            INFO 	Using cached data for _extended_timing_data
C:\Users\prabh\AppData\Local\Temp\ipykernel_10616\910154449.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_data['PitInTimeSec']=laps_data['PitInTime'].dt.total_seconds()
C:\Users\prabh\AppData\Local\Temp\ipykernel_10616\910154449.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_data['PitOutTimeSec']=laps_data['PitOutTime'].dt.total_seconds()


,Driver,LapTime,NumberOfLaps,NumberOfPitStops,PitOutTime,PitInTime,Position,GapToLeader,IntervalToPositionAhead,PitInTimeSec,PitOutTimeSec
0,1,NaT,1.0,0.0,0 days 00:14:44.885000,NaT,NaN,NaN,NaN,NaN,884.885
1,1,0 days 00:01:49.972000,2.0,0.0,NaT,0 days 01:05:25.519000,NaN,NaN,NaN,3925.519,NaN
2,1,0 days 00:01:48.658000,3.0,1.0,0 days 01:05:45.581000,NaT,NaN,NaN,NaN,NaN,3945.581
3,1,0 days 00:01:25.251000,4.0,1.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN
4,1,0 days 00:01:23.461000,5.0,1.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29975,20,NaT,NaN,NaN,NaT,NaT,14.0,+26.521,+0.344,NaN,NaN
29976,20,NaT,NaN,NaN,NaT,NaT,14.0,+23.775,+0.305,NaN,NaN
29977,20,NaT,NaN,NaN,NaT,NaT,14.0,+26.469,+0.300,NaN,NaN
29978,20,NaT,NaN,NaN,NaT,NaT,15.0,+23.775,+0.305,NaN,NaN


### GETTING CORNERS INFO

In [38]:
circuit_info=session.get_circuit_info()
corners_data=circuit_info.corners

In [47]:
master_data = lap_data.merge(position_data, on=['Driver', 'LapNumber'], how='left')
master_data = master_data.merge(tyre_data, on=['Driver', 'LapNumber'], how='left')
master_data = master_data.merge(laps_data, on=['Driver'], how='left')

In [51]:
weather_data

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,0 days 00:00:46.338000,17.4,68.0,1007.4,False,25.5,196,2.7
1,0 days 00:01:46.337000,17.4,67.0,1007.5,False,25.5,193,3.2
2,0 days 00:02:46.336000,17.4,66.0,1007.4,False,25.5,177,3.4
3,0 days 00:03:46.335000,17.5,67.0,1007.4,False,25.6,184,2.4
4,0 days 00:04:46.349000,17.5,67.0,1007.5,False,25.6,186,2.9
...,...,...,...,...,...,...,...,...
204,0 days 03:24:47.183000,15.7,79.0,1008.2,True,19.0,176,3.0
205,0 days 03:25:47.182000,15.8,77.0,1008.2,True,19.1,161,3.0
206,0 days 03:26:47.181000,15.8,77.0,1008.2,True,19.1,163,2.7
207,0 days 03:27:47.180000,15.7,74.0,1008.4,False,18.9,192,3.2


In [49]:
telemetry_data

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Driver
0,2023-08-27 13:03:05.416,10093,0,1,15,False,1,car,0 days 00:00:00.082000,0 days 01:02:05.042000,VER
1,2023-08-27 13:03:05.616,10063,0,1,15,False,1,car,0 days 00:00:00.282000,0 days 01:02:05.242000,VER
2,2023-08-27 13:03:05.897,8517,1,1,15,False,1,car,0 days 00:00:00.563000,0 days 01:02:05.523000,VER
3,2023-08-27 13:03:06.097,7117,11,1,15,False,1,car,0 days 00:00:00.763000,0 days 01:02:05.723000,VER
4,2023-08-27 13:03:06.497,4317,24,1,16,False,1,car,0 days 00:00:01.163000,0 days 01:02:06.123000,VER
...,...,...,...,...,...,...,...,...,...,...,...
601328,2023-08-27 15:27:25.367,11670,294,7,99,False,1,car,0 days 02:24:20.033000,0 days 03:26:24.993000,PIA
601329,2023-08-27 15:27:25.527,11699,295,7,99,False,1,car,0 days 02:24:20.193000,0 days 03:26:25.153000,PIA
601330,2023-08-27 15:27:25.687,11742,295,7,99,False,1,car,0 days 02:24:20.353000,0 days 03:26:25.313000,PIA
601331,2023-08-27 15:27:26.047,11758,296,7,99,False,1,car,0 days 02:24:20.713000,0 days 03:26:25.673000,PIA


In [50]:
master_data

,Driver,LapNumber,LapTime_x,Sector1Time,Sector2Time,Sector3Time,IsAccurate,Position_x,Compound,FreshTyre,...,LapTime_y,NumberOfLaps,NumberOfPitStops,PitOutTime,PitInTime,Position_y,GapToLeader,IntervalToPositionAhead,PitInTimeSec,PitOutTimeSec
0,VER,1.0,0 days 00:01:31.585000,NaT,0 days 00:00:28.929000,0 days 00:00:30.478000,False,1.0,SOFT,True,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,VER,2.0,0 days 00:01:49.972000,0 days 00:00:36.330000,0 days 00:00:36.685000,0 days 00:00:36.957000,False,3.0,SOFT,True,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2,VER,3.0,0 days 00:01:48.658000,0 days 00:00:49.822000,0 days 00:00:30.905000,0 days 00:00:27.931000,False,5.0,INTERMEDIATE,True,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3,VER,4.0,0 days 00:01:25.251000,0 days 00:00:29.996000,0 days 00:00:29.479000,0 days 00:00:25.776000,True,4.0,INTERMEDIATE,True,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
4,VER,5.0,0 days 00:01:23.461000,0 days 00:00:28.478000,0 days 00:00:29.064000,0 days 00:00:25.919000,True,4.0,INTERMEDIATE,True,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,PIA,68.0,0 days 00:01:25.004000,0 days 00:00:29.890000,0 days 00:00:29.060000,0 days 00:00:26.054000,True,9.0,INTERMEDIATE,False,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1339,PIA,69.0,0 days 00:01:22.911000,0 days 00:00:28.834000,0 days 00:00:28.632000,0 days 00:00:25.445000,True,9.0,INTERMEDIATE,False,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1340,PIA,70.0,0 days 00:01:22.659000,0 days 00:00:28.662000,0 days 00:00:28.457000,0 days 00:00:25.540000,True,9.0,INTERMEDIATE,False,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1341,PIA,71.0,0 days 00:01:22.296000,0 days 00:00:28.423000,0 days 00:00:28.419000,0 days 00:00:25.454000,True,9.0,INTERMEDIATE,False,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN


In [ ]:
master_data = master_data.merge(telemetry_data, on=['Driver', 'LapNumber'], how='left')
master_data = master_data.merge(weather_data, on='Time', how='left')